In [3]:
import json
import tensorflow as tf
import numpy as np
from typing import Optional
from src.models.ig_cnn import ig_cnn_model


In [4]:
def gen_conv(filters:int, kernel_size:int, activation:Optional[str]=None, padding:Optional[str]=None):
   _conv = f"C-{filters}-{kernel_size}"
   if activation is not None:
       _conv += f"-{activation}"
   if padding is not None:
       _conv += f"-{padding}"
   return _conv

def gen_dense(units: int, activation: Optional[str]=None):
    _dense = f"D-{units}"
    if  activation is not None:
       _dense += f"-{activation}"
    return _dense

### Generate CNN configs

In [14]:
idx = 0
for phantom_size in [3, 5, 7, 9]:
    for activation in ['sigmoid', 'relu']:
        for filters in [10, 15, 25, 50]:
            for kernel in [3, 5, 7, 9]:
                i = phantom_size
                depth = 0
                layers = ""
                while i >= kernel:
                    depth += 1
                    i -= (kernel-np.floor(kernel/2.0))
                    layers += f"{gen_conv(filters, kernel, activation, padding='valid')},"
                    # Test if model is creatable
                    model_def = f"{layers}"[:-1]
                    model = ig_cnn_model(model_def, input_size=(phantom_size, phantom_size))
                    model.compile(loss='mse')
                    if model.output_shape[-3:] != (1,1,1):
                        continue
                    with open(f"{idx}.json", 'w') as file:
                        config = dict(model_def=model_def,
                                      activation=activation,
                                      kernel=kernel,
                                      filters=filters,
                                      depth=depth,
                                      phantom_size=phantom_size,
                                      log_name=f'{phantom_size}/{model_def}/',
                                      log_root='../logs_cnn/',
                                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                                 )
                        json.dump(config, file)
                    idx += 1

Input 3
C-10-3-sigmoid-valid (1, 1, 1)
C-15-3-sigmoid-valid (1, 1, 1)
C-25-3-sigmoid-valid (1, 1, 1)
C-50-3-sigmoid-valid (1, 1, 1)
C-10-3-relu-valid (1, 1, 1)
C-15-3-relu-valid (1, 1, 1)
C-25-3-relu-valid (1, 1, 1)
C-50-3-relu-valid (1, 1, 1)
Input 5
C-10-3-sigmoid-valid,C-10-3-sigmoid-valid (1, 1, 1)
C-10-5-sigmoid-valid (1, 1, 1)
C-15-3-sigmoid-valid,C-15-3-sigmoid-valid (1, 1, 1)
C-15-5-sigmoid-valid (1, 1, 1)
C-25-3-sigmoid-valid,C-25-3-sigmoid-valid (1, 1, 1)
C-25-5-sigmoid-valid (1, 1, 1)
C-50-3-sigmoid-valid,C-50-3-sigmoid-valid (1, 1, 1)
C-50-5-sigmoid-valid (1, 1, 1)
C-10-3-relu-valid,C-10-3-relu-valid (1, 1, 1)
C-10-5-relu-valid (1, 1, 1)
C-15-3-relu-valid,C-15-3-relu-valid (1, 1, 1)
C-15-5-relu-valid (1, 1, 1)
C-25-3-relu-valid,C-25-3-relu-valid (1, 1, 1)
C-25-5-relu-valid (1, 1, 1)
C-50-3-relu-valid,C-50-3-relu-valid (1, 1, 1)
C-50-5-relu-valid (1, 1, 1)
Input 7
C-10-3-sigmoid-valid,C-10-3-sigmoid-valid,C-10-3-sigmoid-valid (1, 1, 1)
C-10-7-sigmoid-valid (1, 1, 1)
C-15-3-s

### Generate Baseline configs

In [ ]:
idx = 0
for phantom_size in [3, 5]:
    # Test if model is creatable
    model_def = f'dense_baseline_{phantom_size}'
    with open(f"baseline_{phantom_size}.json", 'w') as file:
        config = dict(model_def=model_def,
                      phantom_size=phantom_size,
                      log_name=f'{phantom_size}/baseline/',
                      log_root='../logs/',
                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                 )
        json.dump(config, file)
    idx += 1

Results

In [ ]:
import pandas as pd
import plotly.express as px
from glob import glob

In [ ]:
df = pd.concat([pd.read_csv(f) for f in glob("*.csv")])
df.shape

In [ ]:
df.groupby(['model_def', 'phantom_size'])
df = df.rename(columns={'Information Gain valid': 'ig_max_valid',
                        'Information Gain valid.1': 'ig_mean_valid',
                        'Information Gain train': 'ig_max_train',
                        'Information Gain train.1': 'ig_mean_train',
                        'epoch_loss.1': 'mse_valid'})

In [ ]:
df.columns

In [ ]:
df = df.astype({'phantom_size': 'int32',
                'kernel': 'int32',
                'phantom_sample': 'int32',
                'depth': 'int32',
                'filters': 'int32'})

In [ ]:
(
    df
        .drop(columns=['log_root',
                       'phantom_file_wildcard',
                       'phantom_root',
                       'log_name',
                       'epoch_mean_squared_error',
                       'epoch_mean_squared_error.1',
                       'phantom_sample'])
        .groupby(['model_def', 'phantom_size'])
        .mean()
).sort_values('mse_valid').to_csv('test.csv')

In [ ]:
df.groupby

In [ ]:
px.box(df, x='model_def', y='mse_valid', color='phantom_size')

In [ ]:
df[['kernel', 'phantom_sample', ]]

In [ ]:
df['phantom_size']

In [ ]:
df = df.astype({'phantom_size': 'int32',
                'kernel': 'int32',
                'phantom_sample': 'int32',
                'depth': 'int32',
                'filters': 'int32'})

In [ ]:
(
    df
        .drop(columns=['log_root',
                       'phantom_file_wildcard',
                       'phantom_root',
                       'log_name',
                       'epoch_mean_squared_error',
                       'epoch_mean_squared_error.1',
                       'phantom_sample'])
        .groupby(['model_def', 'phantom_size'])
        .mean()
).sort_values('mse_valid')

In [ ]:
df.groupby

In [ ]:
px.box(df, x='model_def', y='mse_valid', color='phantom_size')

In [ ]:
df[['kernel', 'phantom_sample', ]]

In [ ]:
df['phantom_size']